In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA


In [2]:
folder = PyPDFDirectoryLoader('./census')
documents = folder.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_docs = text_splitter.split_documents(documents)
final_docs[0]

Document(metadata={'source': 'census/acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3')

In [3]:
len(final_docs)

316

In [4]:
embeddings = HuggingFaceEmbeddings(
    model_name = "BAAI/bge-small-en-v1.5",
    model_kwargs = {"device":"cpu"},
    encode_kwargs = {"normalize_embeddings":True}
)

/var/folders/15/vc_j9b113v12tq235z56lnpr0000gn/T/ipykernel_41338/347479446.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/Applications/anaconda3/envs/llm/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
import  numpy as np
print(np.array(embeddings.embed_query(documents[0].page_content)))
print(np.array(embeddings.embed_query(documents[0].page_content)).shape)

[-6.63311332e-02 -1.67243443e-02 -2.26697698e-02  4.53467667e-02
  5.79890981e-02  4.10221443e-02 -3.24309692e-02  1.20888557e-02
 -8.28900039e-02 -4.29334044e-02  4.90752608e-02  4.52029668e-02
 -3.61645781e-02 -2.31252294e-02 -3.28695029e-03  1.86440293e-02
 -6.41976157e-03 -2.78520826e-02 -2.18824651e-02  2.97072437e-02
 -3.06006353e-02  5.17655686e-02  1.94625434e-04 -3.42079364e-02
  2.03822367e-02  2.50208862e-02  4.56858892e-04 -5.42490277e-03
 -5.97405285e-02 -1.50181323e-01 -9.69276647e-04  1.15403458e-02
 -6.36249036e-02 -2.72412300e-02  4.09724889e-03  7.12414912e-04
 -2.51662098e-02  7.83662722e-02  2.48138104e-02  4.11876254e-02
 -3.24607305e-02  1.22553995e-02  3.56448174e-04 -8.12359247e-03
 -4.22629490e-02 -7.11408257e-03 -3.31867039e-02 -2.61397427e-03
  2.31522862e-02 -3.37567739e-02  3.96202393e-02 -3.09692845e-02
  3.46536413e-02  8.34808648e-02  4.80502434e-02 -5.83474897e-02
 -5.68011962e-03 -8.38904362e-03 -3.26032452e-02  7.23538222e-03
  3.79177816e-02  2.47980

In [6]:
db = FAISS.from_documents(final_docs,embeddings)

In [7]:
query="WHAT IS HEALTH INSURANCE COVERAGE?"
q_response = db.similarity_search(query)
print(q_response[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [8]:
db_ret = db.as_retriever(search_type='similarity',search_kwargs={"k":3})
print(db_ret)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x31269f010> search_kwargs={'k': 3}


In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACEHUB_API_TOKEN')

The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

In [11]:
from langchain_community.llms import HuggingFaceHub

In [12]:
hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}
)

/var/folders/15/vc_j9b113v12tq235z56lnpr0000gn/T/ipykernel_41338/3896550165.py:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  hf = HuggingFaceHub(


In [13]:
hf.invoke(query)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'WHAT IS HEALTH INSURANCE COVERAGE?\n\nHealth insurance coverage is a contract between an individual and an insurance company. The individual pays a premium to the insurance company in exchange for the insurance company’s promise to pay for some or all of the individual’s health care costs.\n\nWHAT IS THE AFFORDABLE CARE ACT?\n\nThe Affordable Care Act (ACA) is a federal law that was passed in 2010. The ACA requires most individuals to have health insurance coverage'

# Hugging Face models can be run locally through the HuggingFacePipeline class.

In [14]:
# from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# hf_pipeline = HuggingFacePipeline.from_model_id(
#     model_id="mistralai/Mistral-7B-v0.1",
#     model_kwargs={"temperature":0.1,"max_length":500},
#     task="text-generation"
# )

In [ ]:
# hf_pipeline.invoke(query)

In [16]:
import requests
import os

# Get your Hugging Face token
token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# Make a request to Hugging Face API
headers = {"Authorization": f"Bearer {token}"}
response = requests.get("https://huggingface.co/api/whoami", headers=headers)

# Check if the token is valid
if response.status_code == 200:
    print("Token is valid!")
else:
    print(f"Invalid or expired token. Status Code: {response.status_code}")


Invalid or expired token. Status Code: 401


In [17]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [18]:
from langchain.prompts import PromptTemplate


In [19]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [20]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=db_ret,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [21]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased direct